## IPL Re-predictions

In [2]:
import csv
import math
import time
import itertools
import warnings
import operator
import statistics
import seaborn as sns
import pandas as pd
import numpy as np
# import geopy.distance
import matplotlib.pyplot as plt
from scipy.stats import linregress

import datetime as dt 
from dateutil.parser import parse
from datetime import datetime, timedelta
from scipy import stats
from IPython.display import display_html
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
df_ads = pd.read_csv('gs://aes-analytics-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_V10.5_12102020.csv')
df_ads.reset_index(drop=True, inplace=True)
print(df_ads.shape)
df_ads.head()

(124703, 159)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES,LIVE_OUTAGE,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Weekday,Weekend_flag,Live_outage_group,Priority_Customer_Qty,Dis_From_Live_Centriod_div_Cust_qty,Priority_Dist_Customer_Qty,Dispatch_Location,cldCvrAvg,cldCvrMax,cldCvrMin,dewPtAvg,dewPtMax,dewPtMin,feelsLikeAvg,feelsLikeMax,feelsLikeMin,heatIndexAvg,heatIndexMax,heatIndexMin,mslPresAvg,mslPresMax,mslPresMin,precip,radSolarAvg,radSolarMax,radSolarTot,relHumAvg,relHumMax,relHumMin,sfcPresAvg,sfcPresMax,sfcPresMin,snowfall,spcHumAvg,spcHumMax,spcHumMin,tempAvg,tempMax,tempMin,wetBulbAvg,wetBulbMax,wetBulbMin,windChillAvg,windChillMax,windChillMin,windDir100mAvg,windDir80mAvg,windDirAvg,windSpd100mAvg,windSpd100mMax,windSpd100mMin,windSpd80mAvg,windSpd80mMax,windSpd80mMin,windSpdAvg,windSpdMax,windSpdMin,tempRange,windSpdRange,sfcPresRange,cldCvrRange,relHumRange,relHumRatio,sfcPresRatio,Active_no_of_trucks_per_day,Month,Year,Period
0,2001309371,2000383861.00,3059,BLUE SKY,2007-01-01 03:33:00,2007-01-01 03:48:00,3059.00,SSUB,305.00,4,177.00,1.00,2007-01-01 05:03:00,177.00,False,15.00,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,INDIANAPOLIS,39.88,-86.11,ZONE3,7,3,5,2,1,Marker4,0.71,0.71,1.00,0,0,0,0,0,0,0,0,0,0,0,Monday,False,1,1.00,0.00,1.00,ARL.,92.00,100.00,44.00,33.90,39.50,30.00,30.30,36.70,24.30,38.50,43.70,34.50,1016.40,1026.00,1008.90,0.00,28.50,125.40,684.30,83.30,90.30,79.30,986.80,995.80,980.10,0.00,4.20,5.20,3.60,38.50,43.70,34.50,36.60,41.70,32.80,30.30,36.70,24.30,269.00,269.00,267.00,16.80,20.30,8.80,17.00,20.60,8.90,14.40,17.80,6.70,9.20,11.10,15.70,56.00,11.00,1.14,1.02,nan,1,2007,2007-1
1,2001309371,2000383861.00,3059,BLUE SKY,2007-01-01 03:33:00,2007-01-01 03:48:00,3059.00,SSUB,305.00,4,177.00,1.00,2007-01-01 05:03:00,177.00,False,15.00,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,INDIANAPOLIS,39.88,-86.11,ZONE3,7,3,5,2,1,Marker4,0.71,0.71,1.00,0,0,0,0,0,0,0,0,0,0,0,Monday,False,2,1.00,0.00,1.00,ARL.,92.00,100.00,44.00,33.90,39.50,30.00,30.30,36.70,24.30,38.50,43.7

In [4]:
def ceil_dt(dt, delta):
    return dt + (datetime.min - dt) % delta

def convert_timestamp(func):
    '''
    Decorator to make p1, p2 value a timestamp value
    To perform easy timestamp operations
    '''
    def inner(p1, p2):
        p1 = datetime.strptime(p1, "%Y-%m-%d %H:%M:%S")
        p2 = datetime.strptime(p2, "%Y-%m-%d %H:%M:%S")

        return func(p1, p2)
    return inner

In [5]:
@convert_timestamp
def add_30_mins(start_timestamp, end_timestamp):
    new_timestamp_list = []
    new_timestamp_list.append((ceil_dt(start_timestamp, timedelta(minutes=30))).strftime("%Y-%m-%d %H:%M:%S"))
    new_timestamp = (ceil_dt(start_timestamp, timedelta(minutes=30)))
    while new_timestamp < end_timestamp:
        new_timestamp = new_timestamp + timedelta(minutes=60)
        if new_timestamp > end_timestamp:
            break
        else :
            new_timestamp_list.append(new_timestamp.strftime("%Y-%m-%d %H:%M:%S"))
    
    return new_timestamp_list

In [6]:
def create_time_groups(df):
    time_group_list = []
    for i in range(len(df)):
        time_intervals = add_30_mins(df['CREATION_DATETIME'][i], df['ENERGIZED_DATETIME'][i])
        time_group_list.append(time_intervals)
    
    return time_group_list

time_intervals_group = create_time_groups(df_ads)

In [7]:
list_len = []
for i in range(len(time_intervals_group)):
    list_new = len(time_intervals_group[i])
    list_len.append(list_new)

In [8]:
len(list_len)

124703

In [9]:
final_timestamp = list(itertools.chain(*time_intervals_group))

In [10]:
print(len(list_len))
print(len(final_timestamp))

124703
550381


In [11]:
test = df_ads.copy(deep=True)
test['Times'] = list_len

In [12]:
test_v1 = test.loc[test.index.repeat(test.Times)].reset_index(drop=True)
test_v1['Time_Bucket'] = final_timestamp

In [13]:
test_v1['CREATION_DATETIME'] = pd.to_datetime(test_v1['CREATION_DATETIME'], errors='coerce')
test_v1['ENERGIZED_DATETIME'] = pd.to_datetime(test_v1['ENERGIZED_DATETIME'], errors='coerce')
test_v1['Time_Bucket'] = pd.to_datetime(test_v1['Time_Bucket'], errors='coerce')

test_v1['Time_Since_Outage'] = (test_v1.Time_Bucket - test_v1.CREATION_DATETIME).dt.total_seconds().div(60).round(4)

test_v1['TR'] = (test_v1.ENERGIZED_DATETIME - test_v1.Time_Bucket).dt.total_seconds().div(60).round(4)

In [14]:
test_v1.sort_values(by=['CREATION_DATETIME', 'Time_Bucket'], inplace=True)
test_v1.reset_index(drop=True, inplace=True)
print(test_v1.shape)

(550381, 163)


In [15]:
test_v1['REP_OUTAGE_ID'] = test_v1['OUTAGE_ID'].astype(str) + test_v1['Time_Bucket'].astype(str).str[8:16]
outage_Count_df = test_v1[['OUTAGE_ID','CREATION_DATETIME']].drop_duplicates()

In [16]:
print(list(test_v1.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCU

In [17]:
test_v1.drop(columns= ['Outages_in_last_1hr', 'Outages_in_last_2hr', 'Outages_in_last_3hr', 'Outages_in_last_4hr',
                       'Outages_in_last_5hr', 'Outages_in_last_6hr', 'Outages_in_last_7hr', 'Outages_in_last_8hr',
                       'Outages_in_last_9hr', 'Outages_in_last_10hr'], inplace=True)

In [18]:
test_v1.reset_index(drop=True, inplace=True)
print(test_v1.shape)

(550381, 154)


In [ ]:
# importing the multiprocessing module 


from multiprocessing import Pool
import tqdm
import time

def count_outage(group):
    group = group.reset_index(drop = True)
    df_temp = outage_Count_df[['OUTAGE_ID','CREATION_DATETIME']]
    df_temp['minutes'] = (group['Time_Bucket'][0] - outage_Count_df['CREATION_DATETIME']).dt.total_seconds().div(60)
    df_temp = df_temp[df_temp.minutes > 0]
    df_temp = df_temp[df_temp.OUTAGE_ID != group.OUTAGE_ID[0]]
    group['Outages_in_last_1hr'] = len(df_temp[df_temp.minutes <= 60])
    group['Outages_in_last_2hr'] = len(df_temp[df_temp.minutes <= 120])
    group['Outages_in_last_3hr'] = len(df_temp[df_temp.minutes <= 180])
    group['Outages_in_last_4hr'] = len(df_temp[df_temp.minutes <= 240])
    group['Outages_in_last_5hr'] = len(df_temp[df_temp.minutes <= 300])
    group['Outages_in_last_6hr'] = len(df_temp[df_temp.minutes <= 360])
    group['Outages_in_last_7hr'] = len(df_temp[df_temp.minutes <= 420])
    group['Outages_in_last_8hr'] = len(df_temp[df_temp.minutes <= 480])
    group['Outages_in_last_9hr'] = len(df_temp[df_temp.minutes <= 540])
    group['Outages_in_last_10hr'] = len(df_temp[df_temp.minutes <= 600])
    return group

def grouping_fn(df):
    outage = df.groupby(['REP_OUTAGE_ID'], as_index=False).apply(count_outage)
    return outage
    
if __name__ == '__main__':
    starttime = time.time()
    with Pool(50) as p:
        outage_n_hours =  list(tqdm.tqdm(p.imap(grouping_fn, [test_v1[:10000],test_v1[10000:20000],
                                                              test_v1[20000:30000], test_v1[30000:40000], 
                                                              test_v1[40000:50000], test_v1[50000:60000],
                                                              test_v1[60000:80000], test_v1[80000:85000],
                                                              test_v1[85000:90000],
                                                              test_v1[90000:100000], test_v1[100000:110000],
                                                              test_v1[110000:130000], test_v1[130000:140000],
                                                              test_v1[140000:150000], test_v1[150000:160000], 
                                                              test_v1[160000:170000], test_v1[170000:175000],
                                                              test_v1[175000:180000],
                                                              test_v1[180000:190000], test_v1[190000:200000],
                                                              test_v1[200000:220000], test_v1[220000:230000],
                                                              test_v1[230000:240000], test_v1[240000:250000],
                                                              test_v1[250000:260000], test_v1[260000:280000],
                                                              test_v1[280000:300000], test_v1[300000:320000],
                                                              test_v1[320000:340000], test_v1[340000:350000],
                                                              test_v1[350000:355000], test_v1[355000:360000],
                                                              test_v1[360000:370000], test_v1[370000:380000],
                                                              test_v1[380000:390000], test_v1[390000:400000],
                                                              test_v1[400000:410000], test_v1[410000:420000],
                                                              test_v1[420000:430000], test_v1[430000:440000],
                                                              test_v1[440000:450000], test_v1[450000:460000],
                                                              test_v1[460000:490000], test_v1[490000:500000],
                                                              test_v1[500000:510000], test_v1[510000:520000],
                                                              test_v1[520000:530000], test_v1[530000:540000],
                                                              test_v1[540000:]]), total=100))

 28%|██▊       | 28/100 [27:49<3:51:25, 192.86s/it]

In [21]:
ads_df = pd.concat(outage_n_hours)

In [22]:
print(list(ads_df.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCU

In [23]:
ads_df.shape

(550381, 164)

In [24]:
def filter_low(group):
    group.reset_index(inplace = True)
    if(group.Time_Since_Outage.max() <= 60):
        return group.OUTAGE_ID[0]
    else:
        return 1
outage_id_rem = ads_df.groupby('OUTAGE_ID').apply(filter_low).reset_index(drop = True)

In [25]:
ads_df = ads_df[~ads_df.OUTAGE_ID.isin(outage_id_rem)]
ads_df.reset_index(drop=True, inplace=True)

In [26]:
ads_df[['OUTAGE_ID', 'CREATION_DATETIME', 'Time_Bucket', 'ENERGIZED_DATETIME', 'Time_Since_Outage', 'TR']].head(20)

,OUTAGE_ID,CREATION_DATETIME,Time_Bucket,ENERGIZED_DATETIME,Time_Since_Outage,TR
0,2001309379,2007-01-01 08:16:14,2007-01-01 08:30:00,2007-01-01 10:04:24,13.77,94.40
1,2001309379,2007-01-01 08:16:14,2007-01-01 09:30:00,2007-01-01 10:04:24,73.77,34.40
2,2001309416,2007-01-02 06:45:38,2007-01-02 07:00:00,2007-01-02 08:00:00,14.37,60.00
3,2001309416,2007-01-02 06:45:38,2007-01-02 08:00:00,2007-01-02 08:00:00,74.37,0.00
4,2001309418,2007-01-02 07:12:40,2007-01-02 07:30:00,2007-01-02 09:41:15,17.33,131.25
5,2001309418,2007-01-02 07:12:40,2007-01-02 08:30:00,2007-01-02 09:41:15,77.33,71.25
6,2001309418,2007-01-02 07:12:40,2007-01-02 09:30:00,2007-01-02 09:41:15,137.33,11.25
7,2001309432,2007-01-02 08:54:38,2007-01-02 09:00:00,2007-01-02 10:30:00,5.37,90.00
8,2001309432,2007-01-02 08:54:38,2007-01-02 10:00:00,2007-01-02 10:30:00,65.37,30.00
9,2001309439,2007-01-02 09:14:20,2007-01-02 09:30:00,2007-01-02 13:04:03,15.67,214.05


In [27]:
def feature_add(group):
    group = group.reset_index(drop = True)
    x = [group.Outages_in_last_1hr[0],group.Outages_in_last_2hr[0],group.Outages_in_last_3hr[0],group.Outages_in_last_4hr[0],group.Outages_in_last_5hr[0]
        ,group.Outages_in_last_6hr[0],group.Outages_in_last_7hr[0],group.Outages_in_last_8hr[0],group.Outages_in_last_9hr[0],group.Outages_in_last_10hr[0]]
    
    y = [1,2,3,4,5,6,7,8,9,10]
    
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    group['Slope_outages'] = slope
    
    ROC = [abs(group.Outages_in_last_1hr[0] - group.Outages_in_last_2hr[0]),abs(group.Outages_in_last_2hr[0] - group.Outages_in_last_3hr[0]),
           abs(group.Outages_in_last_3hr[0] - group.Outages_in_last_4hr[0]),abs(group.Outages_in_last_4hr[0] - group.Outages_in_last_5hr[0]),
           abs(group.Outages_in_last_5hr[0] - group.Outages_in_last_6hr[0]),abs(group.Outages_in_last_6hr[0] - group.Outages_in_last_7hr[0]),
           abs(group.Outages_in_last_7hr[0] - group.Outages_in_last_8hr[0]),abs(group.Outages_in_last_8hr[0] - group.Outages_in_last_9hr[0]),
           abs(group.Outages_in_last_9hr[0] - group.Outages_in_last_10hr[0])]
    y = [1,2,3,4,5,6,7,8,9]
    slope, intercept, r_value, p_value, std_err = linregress(ROC, y)
    group['Slope_ROC_outages'] = slope
    
    group['max_ROC_outages'] = max(ROC)
    group['max_index_ROC'] = 1+ROC.index(max(ROC))    
    group['weight_ROC'] = group['max_ROC_outages'] * group['max_index_ROC']*0.1
    return group

In [ ]:
ads_df = ads_df.groupby(['OUTAGE_ID'], as_index = False).apply(feature_add).reset_index(drop = True)

In [29]:
print(list(ads_df.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCU

In [30]:
ads_df.dropna(subset=['Slope_ROC_outages', 'Slope_outages'],inplace=True)
ads_df.drop(['Month', 'Year', 'Period', 'Times', 'Time_Bucket', 'TTR', 'Active_no_of_trucks_per_day', 'EVENT', 'LAT', 'LONG', 'Marker_Location',
              'RANK_SUBSEQUENT_OUTAGES', 'Live_outage_group', 'Dis_From_Live_Centriod_div_Cust_qty'], axis=1, inplace=True)
ads_df.head()

,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DAY_FLAG,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,ZONE,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Hour_Sin,Hour_Cos,LIVE_OUTAGE,Weekday,Weekend_flag,Priority_Customer_Qty,Priority_Dist_Customer_Qty,Dispatch_Location,cldCvrAvg,cldCvrMax,cldCvrMin,dewPtAvg,dewPtMax,dewPtMin,feelsLikeAvg,feelsLikeMax,feelsLikeMin,heatIndexAvg,heatIndexMax,heatIndexMin,mslPresAvg,mslPresMax,mslPresMin,precip,radSolarAvg,radSolarMax,radSolarTot,relHumAvg,relHumMax,relHumMin,sfcPresAvg,sfcPresMax,sfcPresMin,snowfall,spcHumAvg,spcHumMax,spcHumMin,tempAvg,tempMax,tempMin,wetBulbAvg,wetBulbMax,wetBulbMin,windChillAvg,windChillMax,windChillMin,windDir100mAvg,windDir80mAvg,windDirAvg,windSpd100mAvg,windSpd100mMax,windSpd100mMin,windSpd80mAvg,windSpd80mMax,windSpd80mMin,windSpdAvg,windSpdMax,windSpdMin,tempRange,windSpdRange,sfcPresRange,cldCvrRange,relHumRange,relHumRatio,sfcPresRatio,Time_Since_Outage,TR,REP_OUTAGE_ID,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Slope_outages,Slope_ROC_outages,max_ROC_outages,max_index_ROC,weight_ROC
0,2001309379,2000383864.00,494-B/403,2007-01-01 08:16:14,2007-01-01 10:04:24,1252.00,FUSE,125.00,1,5.00,0.00,2007-01-01 11:46:14,5.00,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,INDIANAPOLIS,ZONE4,7,3,5,2,1,0.87,-0.50,0,Monday,False,2.00,2.00,MILL,93.00,100.00,43.00,33.20,39.10,29.50,29.60,36.30,23.50,37.90,43.20,34.00,1016.90,1026.20,1009.30,0.00,29.00,127.20,695.30,83.00,90.00,79.00,986.80,995.70,980.10,0.00,4.10,5.20,3.50,37.90,43.20,34.00,36.00,41.30,32.20,29.60,36.30,23.50,270.00,271.00,268.00,16.80,20.20,8.70,16.90,20.50,8.80,14.40,17.80,6.70,9.20,11.10,15.60,57.00,11.00,1.14,1.02,13.77,94.40,200130937901 08:30,0,0,0,0,1,1,1,1,1,1,5.00,-1.13,1,4,0.40
1,2001309379,2000383864.00,494-B/403,2007-01-01 08:16:14,2007-01-01 10:04:24,1252.00,FUSE,125.00,1,5.00,0.00,2007-01-01 11:46:14,5.00,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,INDIANAPOLIS,ZONE4,7,3,5,2,1,0.87,-0.50,0,Monday,False,2.00,2.00,MILL,93.00,100.00,43.00,33.20,39.10,29.50,29.60,36.30,23.50,37.90,43.20,34.00,1016.90,1026.20,1009.30,0.00,29.00,127.20,695.30,83.00,90.00,79.00,986.80,995.70,980.10,0.00,4.10,5.20,3.50,37.90,43.20,34

In [31]:
ads_df.to_csv('gs://aes-analytics-0002-curated/Outage_Restoration/Repredictions/Master_Dataset/OMS_IPL_Repredictions_V2_12122020.csv')